In [76]:
import psycopg2
import requests
import pandas as pd
from pandas import json_normalize

In [90]:
# Access API with 43 pages

all_data = []

# Loop through pages
for page_num in range(0, 43):
    # Build the url and download the results
    url = 'https://data.bs.ch/api/explore/v2.1/catalog/datasets/100294/records?order_by=timestamp%20ASC&limit=100&offset=' + str(page_num)
    print("Downloading ", url)
    response = requests.get(url)
    data = response.json()
    all_data = all_data + data['results']
    
print("There are", len(all_data), "total results")

There are 4300 total results


In [91]:
# Convert to a data frame
df = pd.DataFrame(all_data)
df

,timestamp,precipitation,relativehumidityhc,solarradiation,airtemperaturehc,windspeedultrasonic,winddirultrasonic
0,2023-04-30T03:00:01+00:00,0.0,86.67,0.0,11.02,0.7,162.0
1,2023-04-30T04:00:01+00:00,0.0,87.35,0.0,10.98,0.7,162.0
2,2023-04-30T05:00:01+00:00,0.0,87.96,3.0,10.93,0.6,160.0
3,2023-04-30T06:00:01+00:00,0.0,89.17,69.0,10.92,0.6,152.0
4,2023-04-30T07:00:01+00:00,0.0,89.50,163.0,11.28,0.8,146.0
...,...,...,...,...,...,...,...
4295,2023-05-05T20:00:01+00:00,0.0,93.36,0.0,16.00,1.3,277.0
4296,2023-05-05T21:00:01+00:00,0.0,95.02,0.0,15.59,0.9,279.0
4297,2023-05-05T22:00:01+00:00,0.0,98.03,0.0,15.29,0.6,276.0
4298,2023-05-05T23:00:01+00:00,0.0,98.89,0.0,14.99,0.5,281.0


In [92]:
try: 
    conn = psycopg2.connect("host=weatherdatalake.ckapktk3je1k.us-east-1.rds.amazonaws.com dbname=weatherdatalake user=nbarnett password=cardsmlb")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

In [93]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)
    
# Auto commit is very important
conn.set_session(autocommit=True)

In [94]:
# Drop Table
cur.execute("DROP TABLE weather")

In [95]:
# Create table in database
cur.execute("CREATE TABLE IF NOT EXISTS weather(timestamp TIMESTAMP, precipitation FLOAT, relativehumidityhc FLOAT, solarradiation FLOAT, airtemperaturehc FLOAT, windspeedultrasonic FLOAT, winddirultrasonic FLOAT);")

In [96]:
# creating column list for insertion
cols = ",".join([str(i) for i in df.columns.tolist()])

In [97]:
# Insert DataFrame recrds one by one.
for i,row in df.iterrows():
    sql = "INSERT INTO weather (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cur.execute(sql, tuple(row))

In [85]:
# cur.execute("INSERT INTO weather(timestamp, precipitation, relativehumidityhc, solarradiation, airtemperaturehc, windspeedultrasonic, winddirultrasonic) VALUES ('2023-06-04T21:00:01+00:00', 0, 73.51, 0, 18.35, 0.6, 220);")

In [98]:
cur.execute("select count(*) from weather;")
print(cur.fetchall())

[(4300,)]


In [87]:
# try: 
#     cur.execute("INSERT INTO weather (timestamp, precipitation, relativehumidityhc, solarradiation, airtemperaturehc, windspeedultrasonic, winddirultrasonic) \
#                  VALUES (%s, %s, %s, %s, %s, %s, %s)", \
#                  ('2023-06-04T21:00:01+00:00', 1.3, 73.51, 0, 18.35, 0.6, 390))
# except psycopg2.Error as e: 
#     print("Error: Inserting Rows")
#     print (e)

In [99]:
try: 
    cur.execute("SELECT * FROM weather;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


(datetime.datetime(2023, 4, 30, 3, 0, 1), 0.0, 86.67, 0.0, 11.02, 0.7, 162.0)
(datetime.datetime(2023, 4, 30, 4, 0, 1), 0.0, 87.35, 0.0, 10.98, 0.7, 162.0)
(datetime.datetime(2023, 4, 30, 5, 0, 1), 0.0, 87.96, 3.0, 10.93, 0.6, 160.0)
(datetime.datetime(2023, 4, 30, 6, 0, 1), 0.0, 89.17, 69.0, 10.92, 0.6, 152.0)
(datetime.datetime(2023, 4, 30, 7, 0, 1), 0.0, 89.5, 163.0, 11.28, 0.8, 146.0)
(datetime.datetime(2023, 4, 30, 8, 0, 1), 0.0, 86.08, 304.0, 11.93, 0.9, 138.0)
(datetime.datetime(2023, 4, 30, 9, 0, 1), 0.0, 78.78, 410.0, 13.23, 0.9, 160.0)
(datetime.datetime(2023, 4, 30, 10, 0, 1), 0.0, 73.09, 565.0, 14.46, 0.9, 121.0)
(datetime.datetime(2023, 4, 30, 11, 0, 1), 0.0, 65.73, 493.0, 15.69, 0.8, 106.0)
(datetime.datetime(2023, 4, 30, 12, 0, 1), 0.0, 61.54, 250.0, 16.51, 0.7, 149.0)
(datetime.datetime(2023, 4, 30, 13, 0, 1), 0.0, 56.39, 319.0, 17.23, 0.9, 158.0)
(datetime.datetime(2023, 4, 30, 14, 0, 1), 0.0, 53.18, 251.0, 17.79, 0.8, 165.0)
(datetime.datetime(2023, 4, 30, 15, 0, 1), 

In [100]:
cur.close()
conn.close()

In [103]:
# url = 'https://data.bs.ch/api/explore/v2.1/catalog/datasets/100294/records?order_by=timestamp%20DESC&limit=1'
# response = requests.get(url)
# data = response.json()
# df = pd.DataFrame(data['results'])
# df

,timestamp,precipitation,relativehumidityhc,solarradiation,airtemperaturehc,windspeedultrasonic,winddirultrasonic
0,2023-10-25T16:00:01+00:00,0.0,54.62,33.0,17.94,2.4,332.0
